<a href="https://colab.research.google.com/github/menosgrande/yahAuct-AVpricechecker/blob/main/yahAuct_AVpricechecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url_site = 'https://erocate.com/search1/q-'
url_mode = '/s-ya/c-ya_2084045794/?o=t1&vmode=1'

headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0"}
cookie = {'adultauthed': '1617782919'}  #クッキーの指定(年齢認証解除)

#LINE送信  
LINE_url = "https://notify-api.line.me/api/notify"
api_token = "wzq0M8MdrUrAqqGAcF5GYVDMdy1hM632xnOkaKQnfL4"
line_headers = {"Authorization": "Bearer " + api_token}

In [ ]:
search_word = input().replace(" ","%20").replace("　","%E3%80%80")# 検索ワード入力

response = requests.get(url=url, headers=headers, cookies=cookie)
html = response.text
soup = BeautifulSoup(html,'lxml') #BeautifulSoupで扱えるようにパースします

url_title = soup.title.string #URLタイトル
mean_price = soup.find(id = 'mean_price')
mean_price_word = "平均:" + mean_price.text + "円" #全体平均

print(url_title + '\n' + url + '\n'+ mean_price_word)

In [ ]:
right_content = soup.find(id = 'right_contents') #右カラムから値段と作品タイトル取得

item_title_list = []#作品値段リスト
for item_title in right_content.find_all(class_ = "item_title"):
   item_title_strip = item_title.text.strip()
   item_title_list.append(item_title_strip)

item_price_list = []#作品値段リスト
for item_price in right_content.find_all(class_ = "item_price"):
   item_price_strip = chr(92) + item_price.text.strip().replace(' 円', '')
   item_price_list.append(item_price_strip)

In [ ]:
test_list = [];#商品リスト
for i, p in enumerate(item_price_list):
    test_list.append({
        "価格": p,
        "タイトル": item_title_list[i],
    });

df = pd.DataFrame(data = test_list)#表にする
css = {
       "background-color": "green",
       "text-align": "right"
      }
display(df.style.set_properties(subset="価格", **css))

In [ ]:
message = search_word +'は' + mean_price_word + '\n' + url
payload = {'message': message}
r = requests.post(LINE_url, headers=line_headers, params=payload)